In [1]:
import pandas as pd

In [2]:
%%time

train = pd.read_parquet('data/data_fusion_train.parquet')[['item_name', 'brands']]

CPU times: user 15.4 s, sys: 8.31 s, total: 23.7 s
Wall time: 20.4 s


In [3]:
%%time

train['count'] = 1
brands_unique = train.groupby('brands').agg({'count': 'count'})
brands_unique = brands_unique[brands_unique['count'] > 1]

CPU times: user 1.22 s, sys: 543 ms, total: 1.77 s
Wall time: 1.59 s


In [4]:
brands_unique.sample(5, random_state=0)

,count
brands,
десант здоровья,72
восточные мотивы,66
peyman,36
weimar,11
harmony,258


In [5]:
brands_unique.shape[0]

15235

In [6]:
import re

def multi_str_re(strings, debug=True, borders=True):
    re_str = '|'.join(
        [re.escape(s) for s in strings]
    )
    if borders:
        re_str = r'\b(?:' + re_str + r')(?!\S)'
        
    if debug:
        print(re_str)
    return re.compile(re_str, re.UNICODE)

brands_re = multi_str_re(
    brands_unique.sort_values(by='count', ascending=False).index, 
    borders=True, 
    debug=False
)

In [7]:
%%time

item_name_unique = train.item_name.drop_duplicates().str.lower().drop_duplicates()

CPU times: user 6.63 s, sys: 785 ms, total: 7.41 s
Wall time: 7.38 s


In [8]:
sample_test = item_name_unique.sample(10000, random_state=0)

In [9]:
item_name_unique.shape, sample_test.shape

((3093158,), (10000,))

In [10]:
def find_all_brands(names_series, brands_re):
    return names_series.str.findall(brands_re)

def find_first_brand(names_series, brands_re):
    brands = find_all_brands(names_series, brands_re)
    first_brand = brands.map(
        lambda b: b[0] if len(b) > 0 else 'отсутствие бренда'
    )
    return first_brand


In [11]:
%%time

find_first_brand(sample_test, brands_re)

CPU times: user 8.39 s, sys: 173 ms, total: 8.57 s
Wall time: 8.47 s


24991406    отсутствие бренда
4114672                  next
23140859    отсутствие бренда
9943678     отсутствие бренда
22744023                  pro
                  ...        
20747703    отсутствие бренда
17935508    отсутствие бренда
6395509     отсутствие бренда
40280218    отсутствие бренда
1806468     отсутствие бренда
Name: item_name, Length: 10000, dtype: object

In [12]:
f"Estimated time for train dataset: {item_name_unique.shape[0] / 10000 * 8.53:.0f} seconds"

'Estimated time for train dataset: 2638 seconds'

In [13]:
from joblib import Parallel, delayed
N_JOBS=8

In [14]:
def find_parallel(names_series, brands_re, n_jobs=N_JOBS):
    n_batches = n_jobs
    batches = [sample_test.iloc[i::n_batches] for i in range(n_batches)]
    brand = Parallel(n_jobs=n_jobs)(
        delayed(find_first_brand)(batch, brands_re)
        for batch in batches
    )
    brand = pd.concat(brand)
    item_brand_mapping = pd.concat([names_series, brand], axis=1, ignore_index=False)
    item_brand_mapping.columns = ['item_name', 'brands']
    return item_brand_mapping

In [15]:
%%time

find_parallel(sample_test, brands_re)

CPU times: user 111 ms, sys: 594 ms, total: 705 ms
Wall time: 2.52 s


,item_name,brands
1093,"баклажаны с орехом состав:орех,чеснок,майонез",отсутствие бренда
1233,макфреш скомбо,отсутствие бренда
1631,пиджак муж. kanzler (54),kanzler
2163,"добрый смузи экзотик рок 0,11л",отсутствие бренда
2782,хурма 1кг,отсутствие бренда
...,...,...
45700978,салат мимоза 350г,отсутствие бренда
45704823,"3. вино эль пасо 0,75л 10-12% бел сух шардоне ...",шардоне
45706914,фасоль стучковая резанная россия 1/10кг,отсутствие бренда
45708526,гун тин шу пуэр,отсутствие бренда


In [16]:
f"Estimated time for train dataset: {item_name_unique.shape[0] / 10000 * 2.75:.0f} seconds"

'Estimated time for train dataset: 851 seconds'

In [17]:
import pickle

pickle.dump(brands_re, open('t2_sub/brands_re', 'wb'))

In [18]:
sample_test = pd.DataFrame(sample_test)
sample_test['id'] = range(sample_test.shape[0])
sample_test.to_parquet('t2_sub/data/task2_test_for_user.parquet')

In [19]:
import zipfile 
compression = zipfile.ZIP_DEFLATED

submission_name = 't2_sub/submission.zip'
with zipfile.ZipFile(submission_name, 'w') as zipObj:
    for filename in [
        'brands_re',
        'script.py',
    ]:
        zipObj.write(
            f't2_sub/{filename}', 
            arcname=filename, 
            compress_type=compression
        )
    print(zipObj.namelist())

!ls -lh {submission_name}

['brands_re', 'script.py']
-rw-r--r-- 1 dmitry.dremov dmitry.dremov 87K фев  4 13:53 t2_sub/submission.zip


# Debug run

In [20]:
%%time

!cd t2_sub; python3 script.py

CPU times: user 85.8 ms, sys: 85.5 ms, total: 171 ms
Wall time: 3.49 s


In [21]:
sample_test.head(10).merge(pd.read_csv('t2_sub/answers.csv'), on='id')

,item_name,id,pred
0,говядина отборная,0,отсутствие бренда
1,next violet mix special 2in1 мрц 99 р(99.00),1,next
2,4. папка уголок а4 150мкм желтая office spase ...,2,отсутствие бренда
3,"вино черный эликсир кр слад болгария 0,75/11%",3,отсутствие бренда
4,клей плиточный усиленный 25кг кeramik pro berg...,4,pro
5,"молоко млада 2,5%ж 1л",5,млада
6,жилет болонь ра325 2д-34 (*5),6,отсутствие бренда
7,конфеты ozera пралине с фундуком кг,7,отсутствие бренда
8,з/паста гелевая дракоша 60мл груша,8,дракоша
9,"влажный корм для котят perfect fit курица, 85 г",9,perfect fit


In [22]:
# Score ?